In [184]:
import sys
sys.path.append('helpers/')

from pprint import pprint
import scipy.io as scio
from eegdProcessors import getEpochsDataFromRaw

In [185]:
eeg_1s_raw = scio.loadmat('./data/subject_1_eeg_separated_epochs_1s.mat')

In [186]:
target_data = getEpochsDataFromRaw(eeg_1s_raw, 0)
nonTarget_data = getEpochsDataFromRaw(eeg_1s_raw, 1)

SETNAME>  EEG target
DATA SHAPE>  (16, 512, 305)
SETNAME>  EEG non-target
DATA SHAPE>  (16, 512, 6447)
